In [1]:
%%capture

# libraries for data manipulation
import pandas as pd
import numpy as np

# libraries for data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

# autoreload modules
%load_ext autoreload
%autoreload 2


# import train_test_split from sklearn
from sklearn.model_selection import train_test_split

# Libraries for Experiment Tracking
import mlflow
import IPython.display


# Libraries for Model Training and Evaluation
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.utils.validation import check_is_fitted

# libraries for file management
import os
import shutil
import joblib
# 
# import data_utils



In [2]:
path = os.path.join(os.getcwd(), "jobs/categories_list.pkl")

# Abrimos el archivo en modo lectura binaria ('rb')
with open(path, 'rb') as f:
    # Cargamos los datos del archivo pickle
    categories_list = pickle.load(f)

categories_list

['PROFESSIONAL_ZIP_3105',
 'PROFESSIONAL_ZIP_3110',
 'PROFESSIONAL_ZIP_3111',
 'PROFESSIONAL_ZIP_3112',
 'PROFESSIONAL_ZIP_3113',
 'PROFESSIONAL_ZIP_3114',
 'PROFESSIONAL_ZIP_3115',
 'PROFESSIONAL_ZIP_3116',
 'PROFESSIONAL_ZIP_3117',
 'PROFESSIONAL_ZIP_3118',
 'PROFESSIONAL_ZIP_3119',
 'PROFESSIONAL_ZIP_3120',
 'PROFESSIONAL_ZIP_3121',
 'PROFESSIONAL_ZIP_3122',
 'PROFESSIONAL_ZIP_3123',
 'PROFESSIONAL_ZIP_3124',
 'PROFESSIONAL_ZIP_3125',
 'PROFESSIONAL_ZIP_3126',
 'PROFESSIONAL_ZIP_3127',
 'PROFESSIONAL_ZIP_3128',
 'PROFESSIONAL_ZIP_3129',
 'PROFESSIONAL_ZIP_3130',
 'PROFESSIONAL_ZIP_3131',
 'PROFESSIONAL_ZIP_3132',
 'PROFESSIONAL_ZIP_3133',
 'PROFESSIONAL_ZIP_3134',
 'PROFESSIONAL_ZIP_3135',
 'PROFESSIONAL_ZIP_3136',
 'PROFESSIONAL_ZIP_3137',
 'PROFESSIONAL_ZIP_3138',
 'PROFESSIONAL_ZIP_3139',
 'PROFESSIONAL_ZIP_3140',
 'PROFESSIONAL_ZIP_3141',
 'PROFESSIONAL_ZIP_3142',
 'PROFESSIONAL_ZIP_3143',
 'PROFESSIONAL_ZIP_3144',
 'PROFESSIONAL_ZIP_3145',
 'PROFESSIONAL_ZIP_3146',
 'PROFESSION

In [3]:
path = os.path.join(os.getcwd(), "jobs/encoding_features.pkl")

# Abrimos el archivo en modo lectura binaria ('rb')
with open(path, 'rb') as f:
    # Cargamos los datos del archivo pickle
    encoding_features = pickle.load(f)

encoding_features

['MARITAL_STATUS',
 'NACIONALITY',
 'RESIDENCE_TYPE',
 'FLAG_EMAIL',
 'FLAG_VISA',
 'FLAG_MASTERCARD',
 'QUANT_BANKING_ACCOUNTS',
 'QUANT_SPECIAL_BANKING_ACCOUNTS',
 'QUANT_CARS',
 'OCCUPATION_TYPE',
 'PRODUCT',
 'PROFESSION_CODE',
 'APPLICATION_SUBMISSION_TYPE',
 'SEX',
 'STATE_OF_BIRTH',
 'CITY_OF_BIRTH',
 'RESIDENCIAL_STATE',
 'RESIDENCIAL_CITY',
 'RESIDENCIAL_BOROUGH',
 'FLAG_RESIDENCIAL_PHONE',
 'RESIDENCIAL_PHONE_AREA_CODE',
 'COMPANY',
 'FLAG_PROFESSIONAL_PHONE',
 'RESIDENCIAL_ZIP_3',
 'PROFESSIONAL_ZIP_3']

In [4]:
path = os.path.join(os.getcwd(), "jobs/num_features_list.pkl")

# Abrimos el archivo en modo lectura binaria ('rb')
with open(path, 'rb') as f:
    # Cargamos los datos del archivo pickle
    num_features_list = pickle.load(f)

num_features_list

['OTHER_INCOMES',
 'PERSONAL_MONTHLY_INCOME',
 'PERSONAL_ASSETS_VALUE',
 'QUANT_DEPENDANTS',
 'MONTHS_IN_RESIDENCE']

In [5]:
path = os.path.join(os.getcwd(), "jobs/ohe_fitted.pkl")

# Abrimos el archivo en modo lectura binaria ('rb')
with open(path, 'rb') as f:
    # Cargamos los datos del archivo pickle
    ohe_fitted = pickle.load(f)

ohe_fitted

OneHotEncoder(drop='first', dtype=<class 'int'>, handle_unknown='ignore',
              sparse_output=False)

In [6]:
path = os.path.join(os.getcwd(), "jobs/remove_features.pkl")

# Abrimos el archivo en modo lectura binaria ('rb')
with open(path, 'rb') as f:
    # Cargamos los datos del archivo pickle
    remove_features = pickle.load(f)

remove_features

['PROFESSIONAL_CITY',
 'PROFESSIONAL_BOROUGH',
 'MATE_PROFESSION_CODE',
 'EDUCATION_LEVEL_2',
 'PROFESSIONAL_STATE',
 'PROFESSIONAL_PHONE_AREA_CODE',
 'QUANT_ADDITIONAL_CARDS',
 'EDUCATION_LEVEL',
 'FLAG_HOME_ADDRESS_DOCUMENT',
 'FLAG_RG',
 'FLAG_CPF',
 'FLAG_INCOME_PROOF',
 'POSTAL_ADDRESS_TYPE',
 'FLAG_DINERS',
 'FLAG_AMERICAN_EXPRESS',
 'FLAG_OTHER_CARDS',
 'MONTHS_IN_THE_JOB',
 'CLERK_TYPE',
 'FLAG_MOBILE_PHONE',
 'FLAG_ACSP_RECORD']

In [7]:
path = os.path.join(os.getcwd(), "jobs/scaler.pkl")

# Abrimos el archivo en modo lectura binaria ('rb')
with open(path, 'rb') as f:
    # Cargamos los datos del archivo pickle
    scaler = pickle.load(f)

scaler

StandardScaler()

In [8]:
path = os.path.join(os.getcwd(), "jobs/train_encoded_dfs.pkl")

# Abrimos el archivo en modo lectura binaria ('rb')
with open(path, 'rb') as f:
    # Cargamos los datos del archivo pickle
    train_encoded_dfs = pickle.load(f)

train_encoded_dfs

[           MARITAL_STATUS2.0  MARITAL_STATUS3.0  MARITAL_STATUS4.0  \
 ID_CLIENT                                                            
 38057                      1                  0                  0   
 35731                      1                  0                  0   
 29839                      0                  0                  0   
 811                        1                  0                  0   
 39136                      0                  0                  0   
 ...                      ...                ...                ...   
 45971                      1                  0                  0   
 5034                       1                  0                  0   
 31417                      1                  0                  0   
 45704                      0                  0                  0   
 21399                      0                  0                  0   
 
            MARITAL_STATUS5.0  MARITAL_STATUS6.0  MARITAL_STATUS7.0  
 ID_C

In [9]:
path = os.path.join(os.getcwd(), "jobs/start_columns.pkl")

# Abrimos el archivo en modo lectura binaria ('rb')
with open(path, 'rb') as f:
    # Cargamos los datos del archivo pickle
    start_columns = pickle.load(f)

start_columns

['PAYMENT_DAY',
 'APPLICATION_SUBMISSION_TYPE',
 'SEX',
 'MARITAL_STATUS',
 'QUANT_DEPENDANTS',
 'STATE_OF_BIRTH',
 'CITY_OF_BIRTH',
 'NACIONALITY',
 'RESIDENCIAL_STATE',
 'RESIDENCIAL_CITY',
 'RESIDENCIAL_BOROUGH',
 'FLAG_RESIDENCIAL_PHONE',
 'RESIDENCIAL_PHONE_AREA_CODE',
 'RESIDENCE_TYPE',
 'MONTHS_IN_RESIDENCE',
 'FLAG_EMAIL',
 'PERSONAL_MONTHLY_INCOME',
 'OTHER_INCOMES',
 'FLAG_VISA',
 'FLAG_MASTERCARD',
 'QUANT_BANKING_ACCOUNTS',
 'QUANT_SPECIAL_BANKING_ACCOUNTS',
 'PERSONAL_ASSETS_VALUE',
 'QUANT_CARS',
 'COMPANY',
 'FLAG_PROFESSIONAL_PHONE',
 'PROFESSION_CODE',
 'OCCUPATION_TYPE',
 'PRODUCT',
 'AGE',
 'RESIDENCIAL_ZIP_3',
 'PROFESSIONAL_ZIP_3']

In [10]:
def convert_payment_day(df):
    # Creamos las nuevas columnas
    for i in range(5, 30, 5):
        df[f'PAYMENT_DAY_{i}'] = df['PAYMENT_DAY'].apply(lambda x: 1 if i-5 < x <= i else 0)
    
    # Agregamos los valores más altos a la columna PAYMENT_DAY_25
    df['PAYMENT_DAY_25'] = df['PAYMENT_DAY'].apply(lambda x: 1 if x >= 25 else 0)

    # Eliminamos la columna original
    df.drop(columns=['PAYMENT_DAY'], inplace=True)
    
    return df


def convert_age(df):
    # Define los rangos de edades que quieres usar
    age_ranges = [(0, 18), (19, 30), (31, 50), (51, 70), (71, 100)]
    
    for i, age_range in enumerate(age_ranges):
        min_age, max_age = age_range
        # Crea una nueva columna para cada rango de edad
        df[f'AGE_RANGE_{min_age}_{max_age}'] = df['AGE'].apply(lambda x: 1 if min_age <= x <= max_age else 0)
    
    # Elimina la columna original de 'AGE'
    df.drop(columns=['AGE'], inplace=True)
    
    return df



In [32]:
json_data = {
  "AGE": "2",
  "APPLICATION_SUBMISSION_TYPE": "Web",
  "CITY_OF_BIRTH": "RIO DE JANEIRO",
  "COMPANY": "Y",
  "FLAG_EMAIL": "0",
  "FLAG_MASTERCARD": "1",
  "FLAG_PROFESSIONAL_PHONE": "N",
  "FLAG_RESIDENCIAL_PHONE": "Y",
  "FLAG_VISA": "0",
  "MARITAL_STATUS": "3",
  "NACIONALITY": "1",
  "OCCUPATION_TYPE": "5",
  "OTHER_INCOMES": "3",
  "PAYMENT_DAY": "4",
  "PERSONAL_ASSETS_VALUE": "50000",
  "PERSONAL_MONTHLY_INCOME": "60000",
  "MONTHS_IN_RESIDENCE": "2",
  "PRODUCT": "2",
  "PROFESSIONAL_ZIP_3": "18",
  "PROFESSION_CODE": "5",
  "QUANT_BANKING_ACCOUNTS": "1",
  "QUANT_CARS": "0",
  "QUANT_DEPENDANTS": "0",
  "QUANT_SPECIAL_BANKING_ACCOUNTS": "1",
  "RESIDENCE_TYPE": "1",
  "RESIDENCIAL_BOROUGH": "1",
  "RESIDENCIAL_CITY": "1",
  "RESIDENCIAL_PHONE_AREA_CODE": "2",
  "RESIDENCIAL_STATE": "1",
  "RESIDENCIAL_ZIP_3": "4",
  "SEX": "F",
  "STATE_OF_BIRTH": "RIO DE JANEIRO"
}

In [33]:
# json to dataframe
df = pd.DataFrame.from_dict(json_data, orient='index', columns=['Value'])

In [34]:
df = df.T

In [37]:
# ordena las columnas de df en funcion de start_columns
df = df[start_columns]

In [15]:
df

,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,STATE_OF_BIRTH,CITY_OF_BIRTH,NACIONALITY,RESIDENCIAL_STATE,RESIDENCIAL_CITY,...,PERSONAL_ASSETS_VALUE,QUANT_CARS,COMPANY,FLAG_PROFESSIONAL_PHONE,PROFESSION_CODE,OCCUPATION_TYPE,PRODUCT,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3
Value,4,Web,F,3,0,RIO DE JANEIRO,RIO DE JANEIRO,1,1,1,...,50000,0,Y,N,5,5,2,2,4,18


In [16]:
#conveert payment day to int
df['PAYMENT_DAY'] = df['PAYMENT_DAY'].astype(int)

In [17]:
df = convert_payment_day(df)
df.shape

(1, 36)

In [26]:
set(new_data.columns)

{'AGE',
 'APPLICATION_SUBMISSION_TYPE',
 'CITY_OF_BIRTH',
 'COMPANY',
 'FLAG_EMAIL',
 'FLAG_MASTERCARD',
 'FLAG_PROFESSIONAL_PHONE',
 'FLAG_RESIDENCIAL_PHONE',
 'FLAG_VISA',
 'MARITAL_STATUS',
 'MONTHS_IN_RESIDENCE',
 'NACIONALITY',
 'OCCUPATION_TYPE',
 'OTHER_INCOMES',
 'PAYMENT_DAY_10',
 'PAYMENT_DAY_15',
 'PAYMENT_DAY_20',
 'PAYMENT_DAY_25',
 'PAYMENT_DAY_5',
 'PERSONAL_ASSETS_VALUE',
 'PERSONAL_MONTHLY_INCOME',
 'PRODUCT',
 'PROFESSIONAL_ZIP_3',
 'PROFESSION_CODE',
 'QUANT_BANKING_ACCOUNTS',
 'QUANT_CARS',
 'QUANT_DEPENDANTS',
 'QUANT_SPECIAL_BANKING_ACCOUNTS',
 'RESIDENCE_TYPE',
 'RESIDENCIAL_BOROUGH',
 'RESIDENCIAL_CITY',
 'RESIDENCIAL_PHONE_AREA_CODE',
 'RESIDENCIAL_STATE',
 'RESIDENCIAL_ZIP_3',
 'SEX',
 'STATE_OF_BIRTH'}

In [27]:
set(start_columns)

{'AGE',
 'APPLICATION_SUBMISSION_TYPE',
 'CITY_OF_BIRTH',
 'COMPANY',
 'FLAG_EMAIL',
 'FLAG_MASTERCARD',
 'FLAG_PROFESSIONAL_PHONE',
 'FLAG_RESIDENCIAL_PHONE',
 'FLAG_VISA',
 'MARITAL_STATUS',
 'MONTHS_IN_RESIDENCE',
 'NACIONALITY',
 'OCCUPATION_TYPE',
 'OTHER_INCOMES',
 'PAYMENT_DAY',
 'PERSONAL_ASSETS_VALUE',
 'PERSONAL_MONTHLY_INCOME',
 'PRODUCT',
 'PROFESSIONAL_ZIP_3',
 'PROFESSION_CODE',
 'QUANT_BANKING_ACCOUNTS',
 'QUANT_CARS',
 'QUANT_DEPENDANTS',
 'QUANT_SPECIAL_BANKING_ACCOUNTS',
 'RESIDENCE_TYPE',
 'RESIDENCIAL_BOROUGH',
 'RESIDENCIAL_CITY',
 'RESIDENCIAL_PHONE_AREA_CODE',
 'RESIDENCIAL_STATE',
 'RESIDENCIAL_ZIP_3',
 'SEX',
 'STATE_OF_BIRTH'}

In [28]:
set(new_data.columns) - set(start_columns)

{'PAYMENT_DAY_10',
 'PAYMENT_DAY_15',
 'PAYMENT_DAY_20',
 'PAYMENT_DAY_25',
 'PAYMENT_DAY_5'}

In [31]:
new_data.columns

Index(['APPLICATION_SUBMISSION_TYPE', 'SEX', 'MARITAL_STATUS',
       'QUANT_DEPENDANTS', 'STATE_OF_BIRTH', 'CITY_OF_BIRTH', 'NACIONALITY',
       'RESIDENCIAL_STATE', 'RESIDENCIAL_CITY', 'RESIDENCIAL_BOROUGH',
       'FLAG_RESIDENCIAL_PHONE', 'RESIDENCIAL_PHONE_AREA_CODE',
       'RESIDENCE_TYPE', 'MONTHS_IN_RESIDENCE', 'FLAG_EMAIL',
       'PERSONAL_MONTHLY_INCOME', 'OTHER_INCOMES', 'FLAG_VISA',
       'FLAG_MASTERCARD', 'QUANT_BANKING_ACCOUNTS',
       'QUANT_SPECIAL_BANKING_ACCOUNTS', 'PERSONAL_ASSETS_VALUE', 'QUANT_CARS',
       'COMPANY', 'FLAG_PROFESSIONAL_PHONE', 'PROFESSION_CODE',
       'OCCUPATION_TYPE', 'PRODUCT', 'AGE', 'RESIDENCIAL_ZIP_3',
       'PROFESSIONAL_ZIP_3', 'PAYMENT_DAY_5', 'PAYMENT_DAY_10',
       'PAYMENT_DAY_15', 'PAYMENT_DAY_20', 'PAYMENT_DAY_25'],
      dtype='object')

In [40]:
import copy

new_data = copy.deepcopy(df)

In [36]:
new_data.shape

(1, 36)

In [41]:
df

,PAYMENT_DAY,APPLICATION_SUBMISSION_TYPE,SEX,MARITAL_STATUS,QUANT_DEPENDANTS,STATE_OF_BIRTH,CITY_OF_BIRTH,NACIONALITY,RESIDENCIAL_STATE,RESIDENCIAL_CITY,...,PERSONAL_ASSETS_VALUE,QUANT_CARS,COMPANY,FLAG_PROFESSIONAL_PHONE,PROFESSION_CODE,OCCUPATION_TYPE,PRODUCT,AGE,RESIDENCIAL_ZIP_3,PROFESSIONAL_ZIP_3
Value,4,Web,F,3,0,RIO DE JANEIRO,RIO DE JANEIRO,1,1,1,...,50000,0,Y,N,5,5,2,2,4,18


In [43]:
encoding_features

['MARITAL_STATUS',
 'NACIONALITY',
 'RESIDENCE_TYPE',
 'FLAG_EMAIL',
 'FLAG_VISA',
 'FLAG_MASTERCARD',
 'QUANT_BANKING_ACCOUNTS',
 'QUANT_SPECIAL_BANKING_ACCOUNTS',
 'QUANT_CARS',
 'OCCUPATION_TYPE',
 'PRODUCT',
 'PROFESSION_CODE',
 'APPLICATION_SUBMISSION_TYPE',
 'SEX',
 'STATE_OF_BIRTH',
 'CITY_OF_BIRTH',
 'RESIDENCIAL_STATE',
 'RESIDENCIAL_CITY',
 'RESIDENCIAL_BOROUGH',
 'FLAG_RESIDENCIAL_PHONE',
 'RESIDENCIAL_PHONE_AREA_CODE',
 'COMPANY',
 'FLAG_PROFESSIONAL_PHONE',
 'RESIDENCIAL_ZIP_3',
 'PROFESSIONAL_ZIP_3']

In [45]:
for feature in encoding_features:
    print(feature)

MARITAL_STATUS
NACIONALITY
RESIDENCE_TYPE
FLAG_EMAIL
FLAG_VISA
FLAG_MASTERCARD
QUANT_BANKING_ACCOUNTS
QUANT_SPECIAL_BANKING_ACCOUNTS
QUANT_CARS
OCCUPATION_TYPE
PRODUCT
PROFESSION_CODE
APPLICATION_SUBMISSION_TYPE
SEX
STATE_OF_BIRTH
CITY_OF_BIRTH
RESIDENCIAL_STATE
RESIDENCIAL_CITY
RESIDENCIAL_BOROUGH
FLAG_RESIDENCIAL_PHONE
RESIDENCIAL_PHONE_AREA_CODE
COMPANY
FLAG_PROFESSIONAL_PHONE
RESIDENCIAL_ZIP_3
PROFESSIONAL_ZIP_3


In [42]:
new_data_encoded_dfs = []

for feature in encoding_features:
    # Usar el codificador entrenado para transformar los nuevos datos
    ohe_new = ohe_fitted.transform(new_data[[feature]])
    # Crear columnas con categorías menos la primera en los nuevos datos
    categories = list(ohe_fitted.categories_[0][1:])
    categories_list = [feature + str(s) for s in categories]

    # Agregar los nuevos DataFrames codificados a las listas
    # Pasar los índices originales al DataFrame
    new_data_encoded_dfs.append(pd.DataFrame(ohe_new, columns=categories_list, index=new_data.index))

# Eliminar la columna original en los nuevos datos
new_data.drop(columns=feature, inplace=True)

# Concatenar todos los DataFrames codificados de una vez
new_data = pd.concat([new_data, *new_data_encoded_dfs], axis=1)

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- APPLICATION_SUBMISSION_TYPE
- COMPANY
- FLAG_EMAIL
- FLAG_MASTERCARD
- FLAG_PROFESSIONAL_PHONE
- ...


In [ ]:
new_data.shape

(1, 35)

In [ ]:
# importar encoding

# remoder --> CITY_OF_BIRTH RESIDENCIAL_CITY RESIDENCIAL_BOROUGH

